In [2]:
import sys
from pathlib import Path

# Add repo root to PYTHONPATH
repo_root = Path.cwd().parents[0]
sys.path.append(str(repo_root))

repo_root

PosixPath('/home/alejandro/Documents/KU_PhD/Projects/EUonAir/claro-library-energy')

In [3]:
from datetime import datetime
import pandas as pd

from src.claro.simulate import SimConfig, simulate, write_csv

In [8]:
cfg = SimConfig(
    seed=123,
    freq_minutes=60,
    n_days=60,
    academic_intensity=1.0,
    building_factor_x4=1.10,
    inertia_phi=0.75
)

rows = simulate(cfg, start=datetime(2025, 1, 1, 0, 0, 0))

from pathlib import Path

# repo root = parent of /notebooks
repo_root = Path.cwd().parents[0]  # works if you're running inside notebooks/
out_path = repo_root / "data" / "simulated" / "claro_simulated.csv"

write_csv(rows, str(out_path))
out_path


PosixPath('/home/alejandro/Documents/KU_PhD/Projects/EUonAir/claro-library-energy/data/simulated/claro_simulated.csv')

In [9]:
from pathlib import Path
Path(out_path).exists()

True

In [7]:
import os
from pathlib import Path

print("CWD (current working directory):", os.getcwd())
p = Path(out_path)

print("Absolute path:", p.resolve())
print("Parent exists?:", p.parent.exists())
print("File exists?:", p.exists())

# List files in the target folder (if it exists)
if p.parent.exists():
    print("Files in data/simulated:", [x.name for x in p.parent.iterdir()])


CWD (current working directory): /home/alejandro/Documents/KU_PhD/Projects/EUonAir/claro-library-energy/notebooks
Absolute path: /home/alejandro/Documents/KU_PhD/Projects/EUonAir/claro-library-energy/notebooks/data/simulated/claro_simulated.csv
Parent exists?: True
File exists?: True
Files in data/simulated: ['claro_simulated.csv']


In [5]:
df = pd.read_csv("data/simulated/claro_simulated.csv")
df.head()


,timestamp,X1_occupancy,X2_temp_out,X2_temp_pressure,X3_open,X4_building_factor,X5_availability,M1_activation,Y_kwh
0,2025-01-01T00:00:00,0.0,4.774,15.226,0.0,1.1,0.77,0.7034,15.8124
1,2025-01-01T01:00:00,0.0,5.124,14.876,0.0,1.1,0.77,0.6873,22.1650
2,2025-01-01T02:00:00,0.0,5.680,14.320,0.0,1.1,0.77,0.6616,28.1995
3,2025-01-01T03:00:00,0.0,6.405,13.595,0.0,1.1,0.77,0.6281,29.4969
4,2025-01-01T04:00:00,0.0,7.249,12.751,0.0,1.1,0.77,0.5891,30.7716


In [6]:
assert (df["Y_kwh"] >= 0).all()

closed = df[df["X3_open"] == 0]
open_ = df[df["X3_open"] == 1]

print("Mean occupancy when closed:", closed["X1_occupancy"].mean())
print("Mean Y when open:", open_["Y_kwh"].mean())
print("Mean Y when closed:", closed["Y_kwh"].mean())

df[["Y_kwh","X1_occupancy","X2_temp_pressure","M1_activation"]].corr()


Mean occupancy when closed: 0.0
Mean Y when open: 76.00703834688348
Mean Y when closed: 44.08146011396011


,Y_kwh,X1_occupancy,X2_temp_pressure,M1_activation
Y_kwh,1.000000,0.749244,0.086317,0.767106
X1_occupancy,0.749244,1.000000,-0.387987,0.916469
X2_temp_pressure,0.086317,-0.387987,1.000000,-0.305709
M1_activation,0.767106,0.916469,-0.305709,1.000000
